# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

### List all the team members BITS ID ,Name along with % of contribution in this assignment: sample Provided below:

| Sr. No. | BITS ID | NAME | % of Contribution |
|----|-----|-------|-----|
| 1 | 2024AA05899 | SHAH BHAVIN ARVINDKUMAR | 33.33% |
| 2 | 2024AA05896 | ANIL PRASAD CHELAMCHELA | 33.33% |
| 3 | 2024AA05897 | SONAAL KALRA | 33.33% |
| 4 |  | DHEERAJ SINGH CHAUHAN | % |
| 5 |  | KAUSTAV BANERJEE | % |


Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

Its mentioned in word document as per the instruction in problem statement.

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [76]:
#Code Block : Set Initial State (Must handle dynamic inputs)
import heapq
import random
import time
import sys
import tracemalloc  # For measuring memory usage

# Road network with travel times (costs)
roads = {
    'A': {'B': 15, 'D1': 30},
    'B': {'A': 15, 'C': 11, 'D2': 22},
    'C': {'B': 11, 'G': 9},
    'D1': {'A': 30, 'G': 4, 'E': 12, 'F': 6},
    'D2': {'B': 22, 'G': 19},
    'E': {'F': 27, 'D1': 12},
    'F': {'E': 27, 'D1': 6, 'H': 20, 'X': 18},
    'G': {'C': 9, 'D1': 4, 'D2': 19, 'X': 17},
    'H': {'F': 20},
    'X': {'F': 18, 'G': 17}
}

In [98]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)
# Function to find all paths from start to end city
def find_all_paths(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if node not in path or node == end:  # Allow revisiting nodes only if they are the end node
            new_paths = find_all_paths(graph, node, end, path)
            for p in new_paths:
                paths.append(p)
    # print(start, end, paths)
    return paths

# Function to calculate travel time for a path
def travel_time(graph, path):
    time = 0
    for i in range(len(path) - 1):
        time += graph[path[i]][path[i+1]]
    return time

# Heuristic function for a specific end city
def heuristic_for_all_nodes(graph, end):
    heuristics = {}
    for start in graph:
        paths = find_all_paths(graph, start, end)
        if paths:
            total_time = sum(travel_time(graph, path) for path in paths)
            heuristics[start] = total_time / len(paths)
        else:
            heuristics[start] = float('inf')  # No path available
    return heuristics

def fitness(route, graph):
    travel_time = 0
    for i in range(len(route) - 1):
        if route[i + 1] in graph[route[i]]:
            travel_time += graph[route[i]][route[i + 1]]
        else:
            return float('inf')  # Invalid route, assign a high travel time
    return travel_time

In [78]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented
def successors(graph, node):
    return graph[node].items()

In [79]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented
def is_goal(node, goal):
    return node == goal

### 2.	Definition of Algorithm 1 (A* Algorithm)

In [80]:
#Code Block : Function for A* algorithm implementation
def a_star_search(graph, start, goal, heuristics):
    open_list = []
    heapq.heappush(open_list, (0, start))
    came_from = {}
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0
    f_score = {node: float('inf') for node in graph}
    f_score[start] = heuristics[start]

    tracemalloc.start()
    start_time = time.time()
    # print("start_time", start_time)
    
    while open_list:
        current = heapq.heappop(open_list)[1]
        
        if is_goal(current, goal):
            path = reconstruct_path(came_from, current)
            current, peak = tracemalloc.get_traced_memory()
            end_time = time.time()
            # print("end_time", end_time)
            tracemalloc.stop()
            return path, g_score[goal], current, peak, end_time - start_time
        
        for neighbor, cost in successors(graph, current):
            tentative_g_score = g_score[current] + cost
            
            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristics[neighbor]
                if neighbor not in [i[1] for i in open_list]:
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))

    tracemalloc.stop()
    return None, float('inf'), None, None, None

def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path


### 3.	Definition of Algorithm 2 (Genetic Algorithm)

In [81]:
#Code Block : Function for Genetic algorithm implementation
def generate_route(cities):
    return random.sample(cities, len(cities))

def initial_population(size, cities):
    return [generate_route(cities) for _ in range(size)]

def select_parents(population, fitnesses, k=3):
    selected = random.choices(population, k=k)
    selected.sort(key=lambda x: fitness(x, roads))
    return selected[0]

def crossover(parent1, parent2):
    cut = random.randint(1, len(parent1) - 2)
    child1 = parent1[:cut] + [gene for gene in parent2 if gene not in parent1[:cut]]
    child2 = parent2[:cut] + [gene for gene in parent1 if gene not in parent2[:cut]]
    return child1, child2

def mutate(route, mutation_rate=0.01):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(route)), 2)
        route[i], route[j] = route[j], route[i]
    return route


def genetic_algorithm_with_metrics(graph, cities, population_size=100, generations=500, mutation_rate=0.01):
    tracemalloc.start()
    start_time = time.time()
    
    population = initial_population(population_size, cities)
    for _ in range(generations):
        population_fitness = [(route, fitness(route, graph)) for route in population]
        population_fitness.sort(key=lambda x: x[1])
        population = [route for route, _ in population_fitness[:population_size // 2]]
        
        next_generation = []
        while len(next_generation) < population_size:
            parent1 = select_parents(population, population_fitness)
            parent2 = select_parents(population, population_fitness)
            child1, child2 = crossover(parent1, parent2)
            next_generation.append(mutate(child1, mutation_rate))
            next_generation.append(mutate(child2, mutation_rate))
        
        population = next_generation
    
    best_route = min(population, key=lambda x: fitness(x, graph))
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    return best_route, fitness(best_route, graph), current, peak, end_time - start_time




### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [88]:
#Code Block : Function & call to get inputs (start/end state)
start = input("Enter the starting city from A, B, C, D1, D2, E, F, G, H, X: ")
goal = input("Enter the destination city from A, B, C, D1, D2, E, F, G, H, X: ")

Enter the starting city from A, B, C, D1, D2, E, F, G, H, X:  A
Enter the destination city from A, B, C, D1, D2, E, F, G, H, X:  X


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [89]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
heuristic = heuristic_for_all_nodes(roads, goal)
path, cost, current_mem, peak_mem, exec_time = a_star_search(roads, start, goal, heuristics)

print(f"Path: {path}")
print(f"Cost: {cost}")

Path: ['A', 'D1', 'F', 'X']
Cost: 54


In [105]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
cities = list(roads.keys())
best_route, best_fitness, gcurrent_mem, gpeak_mem, gexec_time = genetic_algorithm_with_metrics(roads, cities)

print(f"Best Route: {best_route}")
print(f"Total Travel Time (Fitness): {best_fitness} units")


Best Route: ['D2', 'X', 'G', 'B', 'H', 'F', 'E', 'C', 'D1', 'A']
Total Travel Time (Fitness): inf units


In [106]:
#Code Block : Print the Time & Space complexity of algorithm 1 - A* Algorithm

print(f"Current memory usage: {current_mem} bytes")
print(f"Peak memory usage: {peak_mem} bytes")
print(f"Execution time: {exec_time} seconds")

Current memory usage: 424 bytes
Peak memory usage: 648 bytes
Execution time: 0.0 seconds


In [107]:
#Code Block : Print the Time & Space complexity of algorithm 2 - Genetic Algorithm

print(f"Current memory usage: {gcurrent_mem} bytes")
print(f"Peak memory usage: {gpeak_mem} bytes")
print(f"Execution time: {gexec_time} seconds")

Current memory usage: 32206 bytes
Peak memory usage: 33422 bytes
Execution time: 1.289045810699463 seconds


### 5.	Comparitive Analysis (Time and Space Complexity)

In [108]:
print(f"A* Path: {path}")
print(f"A* Cost: {cost}")
print(f"A* Current memory usage: {current_mem} bytes")
print(f"A* Peak memory usage: {peak_mem} bytes")
print(f"A* Execution time: {exec_time} seconds")
print(f"Genetic Algorithm Best Route: {best_route}")
print(f"Genetic Algorithm Total Travel Time (Fitness): {best_fitness} units")
print(f"Genetic Algorithm Current memory usage: {gcurrent_mem} bytes")
print(f"Genetic Algorithm Peak memory usage: {gpeak_mem} bytes")
print(f"Genetic Algorithm Execution time: {gexec_time} seconds")

A* Path: ['A', 'D1', 'F', 'X']
A* Cost: 54
A* Current memory usage: 424 bytes
A* Peak memory usage: 648 bytes
A* Execution time: 0.0 seconds
Genetic Algorithm Best Route: ['D2', 'X', 'G', 'B', 'H', 'F', 'E', 'C', 'D1', 'A']
Genetic Algorithm Total Travel Time (Fitness): inf units
Genetic Algorithm Current memory usage: 32206 bytes
Genetic Algorithm Peak memory usage: 33422 bytes
Genetic Algorithm Execution time: 1.289045810699463 seconds


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : A* algorithm is more efficient in terms of both time and space complexity compared to the genetic algorithm. Genetic algorithms, although useful for global optimization, generally consume more memory and time due to their iterative and population-based nature.